# Studi Kasus Naive Bayes 1

## Analisa sentimen bahasa indonesia menggunakan Multinominal Naive Bayes

Step 1. Pembersihan data (Pre-processing)

In [4]:
#import regex
import re
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary


Buat Fungsi penghapusan

    URLs
    Hashtags
    Mentions
    Emojis dan  Smileys
    Stemming
    Remove Stopword dan Reserved words (RT, FAV)


In [5]:
#start process_tweet
def preprocessTweet(tweet):
    #konversi menjadi huruf kecil
    tweet = tweet.lower()
    #konversi www.* atau https?://* menjadi URL
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet)
    
    #Ganti #tagar menjadi tagar
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    
    #konversi mention @username mejadi AT_USER
    tweet = re.sub('@[^\s]+','AT_USER',tweet)
     # keep only words
    tweet = re.sub("[^a-zA-Z]", " ", tweet) 
    #Buang  white spaces
    tweet = re.sub('[\s]+', ' ', tweet)

    #trim
    tweet = tweet.strip('\'"')
  
    #setelah bersih di stem
    stemmerFactory = StemmerFactory()
    stemmer = stemmerFactory.create_stemmer()
    tweet   = stemmer.stem(tweet)
    
    
    # Ambil Stopword bawaan
    stopwordDefault = StopWordRemoverFactory().get_stop_words()
    stopwordTambahan = ['RT', 'lol']

    # Merge stopword
    dataStopword = stopwordDefault + stopwordTambahan

    dictionary = ArrayDictionary(dataStopword)
    stopRemover = StopWordRemover(dictionary)

    tweet = (stopRemover.remove(tweet))


    return tweet
#end


Ujicoba Preprosessing

In [6]:
tw1="Tuhan sangat cinta negeri indah ini. Tuhan selalu mengirimkan pemimpin yg hebat. Dukungan penuh dari rakyatlah yg akan memampukan pemerintahan Jokowi untuk terus membangun. Mari kita dukung beliau dgn sepenuh hati. #JokowiMaruf2019 #jokowi2019sekalilagi #TetapJokowi"
print (tw1,"\n\n")
print (preprocessTweet(tw1))

Tuhan sangat cinta negeri indah ini. Tuhan selalu mengirimkan pemimpin yg hebat. Dukungan penuh dari rakyatlah yg akan memampukan pemerintahan Jokowi untuk terus membangun. Mari kita dukung beliau dgn sepenuh hati. #JokowiMaruf2019 #jokowi2019sekalilagi #TetapJokowi 


tuhan cinta negeri indah tuhan kirim pimpin yg hebat dukung penuh rakyat yg perintah jokowi bangun mari dukung beliau dgn sepenuh hati jokowimaruf jokowi sekalilagi tetapjokowi


In [7]:

#Baca data tweet
fp = open('data/dataset.csv', 'r')
line = fp.readline()

while line:
    processedTweet = preprocessTweet(line)
    print (processedTweet)
    line = fp.readline()
#end loop
fp.close()

positive tuhan cinta negeri indah tuhan kirim pimpin yg hebat dukung penuh rakyat yg perintah jokowi bangun mari dukung beliau dgn sepenuh hati jokowimaruf jokowi sekalilagi tetapjokowi
negatif binatang pegang tali manusia pegang janji bunyi pepatah janji sekira sanggup penuh janji capres ri realisasi janji hutang mudah umbar janji


In [26]:
class BagOfWords(object):
    
    def __init__(self):
        self.__bag_of_words = {}
        
    def add(self,word):
        if word in self.__bag_of_words:
            self.__bag_of_words[word] += 1
        else:
            self.__bag_of_words[word] = 1
    
    def len(self):
        """ Returning the number of different words of an object """
        return len(self.__bag_of_words)
    
    def Words(self):
        """ Returning a list of the words contained in the object """
        return self.__bag_of_words.keys()
    
        
    def BagOfWords(self):
        """ Returning the dictionary, containing the words (keys) with their frequency (values)"""
        return self.__bag_of_words
        
    def WordFreq(self,word):
        """ Returning the frequency of a word """
        if word in self.__bag_of_words:
            return self.__bag_of_words[word]
        else:
            return 0

In [27]:
kata2 = BagOfWords()
kata2.add("kucing")
kata2.add("kuda")
kata2.add("anjing")
kata2.add("kucing")

print (kata2.BagOfWords())
print (kata2.Words())


{'kucing': 2, 'kuda': 1, 'anjing': 1}
dict_keys(['kucing', 'kuda', 'anjing'])


In [37]:
class Document(object):
    """ Used both for learning (training) documents and for testing documents. The optional parameter lear
    has to be set to True, if a classificator should be trained. If it is a test document learn has to be set to False. """
    _vocabulary = BagOfWords()
 
    def __init__(self, vocabulary):
        self.__name = ""
        self.__document_class = None
        self._words_and_freq = BagOfWords()
        Document._vocabulary = vocabulary
    
    def read_document(self,filename, learn=False):
        """ A document is read. It is assumed that the document is either encoded in utf-8 or in iso-8859... (latin-1).
        The words of the document are stored in a Bag of Words, i.e. self._words_and_freq = BagOfWords() """
        try:
            text = open(filename,"r", encoding='utf-8').read()
        except UnicodeDecodeError:
            text = open(filename,"r", encoding='latin-1').read()
        text = text.lower()
        self.set_document(text,learn)
        
    def set_document(self,text, learn=False):
        words = re.split(r"\W",text)

        self._number_of_words = 0
        for word in words:
            self._words_and_freq.add(word)
            if learn:
                Document._vocabulary.add(word)


    def vocabulary_length(self):
        """ Returning the length of the vocabulary """
        return len(Document._vocabulary)
                
    def WordsAndFreq(self):
        """ Returning the dictionary, containing the words (keys) with their frequency (values) as contained
        in the BagOfWords attribute of the document"""
        return self._words_and_freq.BagOfWords()
        
    def Words(self):
        """ Returning the words of the Document object """
        d =  self._words_and_freq.BagOfWords()
        return d.keys()
    
    def WordFreq(self,word):
        """ Returning the number of times the word "word" appeared in the document """
        bow =  self._words_and_freq.BagOfWords()
        if word in bow:
            return bow[word]
        else:
            return 0
                
    def __and__(self, other):
        """ Intersection of two documents. A list of words occuring in both documents is returned """
        intersection = []
        words1 = self.Words()
        for word in other.Words():
            if word in words1:
                intersection += [word]
        return intersection

In [40]:
doku = Document("positif")
doku.set_document(processedTweet)
print(doku.WordsAndFreq())

{'negatif': 1, 'binatang': 1, 'pegang': 2, 'tali': 1, 'manusia': 1, 'janji': 5, 'bunyi': 1, 'pepatah': 1, 'sekira': 1, 'sanggup': 1, 'penuh': 1, 'capres': 1, 'ri': 1, 'realisasi': 1, 'hutang': 1, 'mudah': 1, 'umbar': 1}


In [29]:
class DocumentClass(Document):
    def __init__(self, vocabulary):
        Document.__init__(self, vocabulary)
        self._number_of_docs = 0

    def Probability(self,word):
        """ returns the probabilty of the word "word" given the class "self" """
        voc_len = Document._vocabulary.len()
        SumN = 0
        for i in range(voc_len):
            SumN = DocumentClass._vocabulary.WordFreq(word)
        N = self._words_and_freq.WordFreq(word)
        erg = 1 + N
        erg /= voc_len + SumN
        return erg

    def __add__(self,other):
        """ Overloading the "+" operator. Adding two DocumentClass objects consists in adding the 
        BagOfWords of the DocumentClass objectss """
        res = DocumentClass(self._vocabulary)
        res._words_and_freq = self._words_and_freq + other._words_and_freq 
 
        return res

    def SetNumberOfDocs(self, number):
        self._number_of_docs = number
    
    def NumberOfDocuments(self):
        return self._number_of_docs

In [ ]:
doku_kelas = DocumentClass()

In [30]:
class Pool(object):
    def __init__(self):
        self.__document_classes = {}
        self.__vocabulary = BagOfWords()
            
    def sum_words_in_class(self, dclass):
        """ The number of times all different words of a dclass appear in a class """
        sum = 0
        for word in self.__vocabulary.Words():
            WaF = self.__document_classes[dclass].WordsAndFreq()
            if word in WaF:
                sum +=  WaF[word]
        return sum
    
    def learn(self, directory, dclass_name):
        """ directory is a path, where the files of the class with the name dclass_name can be found """
        x = DocumentClass(self.__vocabulary)
        dir = os.listdir(directory)
        for file in dir:
            d = Document(self.__vocabulary)
            print(directory + "/" + file)
            d.read_document(directory + "/" +  file, learn = True)
            x = x + d
        self.__document_classes[dclass_name] = x
        x.SetNumberOfDocs(len(dir))

    
    def Probability(self, doc, dclass = ""):
        """Calculates the probability for a class dclass given a document doc"""
        if dclass:
            sum_dclass = self.sum_words_in_class(dclass)
            prob = 0
        
            d = Document(self.__vocabulary)
            d.read_document(doc)

            for j in self.__document_classes:
                sum_j = self.sum_words_in_class(j)
                prod = 1
                for i in d.Words():
                    wf_dclass = 1 + self.__document_classes[dclass].WordFreq(i)
                    wf = 1 + self.__document_classes[j].WordFreq(i)
                    r = wf * sum_dclass / (wf_dclass * sum_j)
                    prod *= r
                prob += prod * self.__document_classes[j].NumberOfDocuments() / self.__document_classes[dclass].NumberOfDocuments()
            if prob != 0:
                return 1 / prob
            else:
                return -1
        else:
            prob_list = []
            for dclass in self.__document_classes:
                prob = self.Probability(doc, dclass)
                prob_list.append([dclass,prob])
            prob_list.sort(key = lambda x: x[1], reverse = True)
            return prob_list

    def DocumentIntersectionWithClasses(self, doc_name):
        res = [doc_name]
        for dc in self.__document_classes:
            d = Document(self.__vocabulary)
            d.read_document(doc_name, learn=False)
            o = self.__document_classes[dc] &  d
            intersection_ratio = len(o) / len(d.Words())
            res += (dc, intersection_ratio)
        return res

In [32]:
DClasses = ["clinton",  "lawyer",  "math",  "medical",  "music",  "sex"]

base = "data/"
p = Pool()
for i in DClasses:
    p.learn(base + i, i)



NameError: name 'os' is not defined